In [3]:
# 3번 N그램에금리 붙이기

import numpy as np
import pandas as pd
import csv
from datetime import datetime, timedelta

### 금리데이터 모으기 
- 한국은행 경제통계 시스템에서 csv파일 다운로드(콜금리, 기준금리: 2004-11-01부터 2017-12-31까지)
(http://ecos.bok.or.kr/flex/EasySearch.jsp?langGubun=K&topCode=060Y001)

In [5]:
call_rate = pd.read_csv('CallRate_1.csv', header=None)
call_rate.columns = ['date', 'call_rate']
call_rate['date'] = call_rate['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) #str형식의 date를 date형식으로 변환
call_rate['call_rate'] = call_rate['call_rate'].fillna(method = 'ffill') #결측치는 가장 가까운 전날의 금리로 채운다
call_rate

,date,call_rate
0,2004-11-01,3.53
1,2004-11-02,3.53
2,2004-11-03,3.53
3,2004-11-04,3.53
4,2004-11-05,3.52
...,...,...
4804,2017-12-27,1.51
4805,2017-12-28,1.55
4806,2017-12-29,1.58
4807,2017-12-30,1.58


In [6]:
standard_rate = pd.read_csv('standard_rate.csv', header=None)
standard_rate.columns = ['date', 'standard_rate']
standard_rate['date'] = standard_rate['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) #str형식의 date를 date형식으로 변환
standard_rate #결측치 없이 제공되므로 그대로 사용

FileNotFoundError: [Errno 2] File standard_rate.csv does not exist: 'standard_rate.csv'

In [5]:
rates = pd.merge(call_rate, standard_rate, on='date')
rates

,date,call_rate,standard_rate
0,2004-11-01,3.53,3.5
1,2004-11-02,3.53,3.5
2,2004-11-03,3.53,3.5
3,2004-11-04,3.53,3.5
4,2004-11-05,3.52,3.5
...,...,...,...
4804,2017-12-27,1.51,1.5
4805,2017-12-28,1.55,1.5
4806,2017-12-29,1.58,1.5
4807,2017-12-30,1.58,1.5


### 크롤링한 데이터에 날짜(당일, 1달 전) 붙이기 
- 같은 요일의 날짜로 확인하기 위해 1달 전을 28일로 정함 

#### 의사록

In [119]:
minutes = pd.read_csv('minutes_ngram_date.csv', header=0, index_col=[0])
minutes['date'] = minutes['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) #str형식의 date를 date형식으로 변환
minutes

,date,token_ngram
0,2005-07-07,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/..."
1,2005-08-11,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG..."
2,2005-09-08,"금융/NNG,불안사태/NNG,발생/NNG,하/VV,가운데/NNG,국제/NNG,환투기..."
3,2005-10-11,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N..."
4,2005-11-10,"미/NNG,달러/NNG,엔/NNG,약세/NNG,지속/NNG,되/VV,보/VV,물/V..."
...,...,...
141,2017-05-25,"국제/NNG,금융시장/NNG,안정/NNG,되/VV,지속/NNG,하/XSV,가운데/N..."
142,2017-07-13,"성장/NNG,고용/NNG,회복국면/NNG,정점/NNG,향하/VV,불구/NNG,물가/..."
143,2017-08-31,"금리정상화/NNG,행보/NNG,불구/NNG,미/NNG,달러/NNG,약세/NNG,기조..."
144,2017-10-19,"임금/NNG,상승/NNG,확대/NNG,되/XSV,세제/NNG,개혁/NNG,기대/NN..."


In [120]:
call_rate.columns = ['date', 'call_rate']
minutes_w_rate = pd.merge(minutes, call_rate, how = 'left', on='date')
minutes_w_rate

,date,token_ngram,call_rate
0,2005-07-07,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/...",3.03
1,2005-08-11,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG...",3.26
2,2005-09-08,"금융/NNG,불안사태/NNG,발생/NNG,하/VV,가운데/NNG,국제/NNG,환투기...",3.24
3,2005-10-11,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N...",3.50
4,2005-11-10,"미/NNG,달러/NNG,엔/NNG,약세/NNG,지속/NNG,되/VV,보/VV,물/V...",3.49
...,...,...,...
141,2017-05-25,"국제/NNG,금융시장/NNG,안정/NNG,되/VV,지속/NNG,하/XSV,가운데/N...",1.22
142,2017-07-13,"성장/NNG,고용/NNG,회복국면/NNG,정점/NNG,향하/VV,불구/NNG,물가/...",1.24
143,2017-08-31,"금리정상화/NNG,행보/NNG,불구/NNG,미/NNG,달러/NNG,약세/NNG,기조...",1.27
144,2017-10-19,"임금/NNG,상승/NNG,확대/NNG,되/XSV,세제/NNG,개혁/NNG,기대/NN...",1.24


In [121]:
call_rate.columns = ['before_month', 'call_rate']
minutes_w_rate['before_month'] = minutes_w_rate['date'].apply(lambda x: x - timedelta(days=28)) #한달 전 날짜 생성해서 콜금리와 병합
minutes_w_rate = pd.merge(minutes_w_rate, call_rate, how = 'left', on='before_month')
minutes_w_rate

,date,token_ngram,call_rate_x,before_month,call_rate_y
0,2005-07-07,"들/VV,국제/NNG,유가/NNG,크/VA,오르/VV,원화/NNG,엔/NNG,화간/...",3.03,2005-06-09,3.28
1,2005-08-11,"위안/NNG,절상/NNG,원화/NNG,절상/NNG,가장/MAG,크/VA,추가/NNG...",3.26,2005-07-14,3.27
2,2005-09-08,"금융/NNG,불안사태/NNG,발생/NNG,하/VV,가운데/NNG,국제/NNG,환투기...",3.24,2005-08-11,3.26
3,2005-10-11,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N...",3.50,2005-09-13,3.24
4,2005-11-10,"미/NNG,달러/NNG,엔/NNG,약세/NNG,지속/NNG,되/VV,보/VV,물/V...",3.49,2005-10-13,3.49
...,...,...,...,...,...
141,2017-05-25,"국제/NNG,금융시장/NNG,안정/NNG,되/VV,지속/NNG,하/XSV,가운데/N...",1.22,2017-04-27,1.17
142,2017-07-13,"성장/NNG,고용/NNG,회복국면/NNG,정점/NNG,향하/VV,불구/NNG,물가/...",1.24,2017-06-15,1.23
143,2017-08-31,"금리정상화/NNG,행보/NNG,불구/NNG,미/NNG,달러/NNG,약세/NNG,기조...",1.27,2017-08-03,1.22
144,2017-10-19,"임금/NNG,상승/NNG,확대/NNG,되/XSV,세제/NNG,개혁/NNG,기대/NN...",1.24,2017-09-21,1.25


In [7]:
minutes_w_rate['change'] = minutes_w_rate['call_rate_x'] - minutes_w_rate['call_rate_y']
minutes_w_rate_3bp = minutes_w_rate

minutes_w_rate['label'] = minutes_w_rate['change'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0 ) #threshold 3bp를 적용하지 않으면 114개 남음
minutes_w_rate = minutes_w_rate[minutes_w_rate['label'] != 0]
minutes_w_rate = minutes_w_rate[['token_ngram', 'label']]
minutes_w_rate

NameError: name 'minutes_w_rate' is not defined

In [123]:
minutes_w_rate_3bp['label'] = minutes_w_rate_3bp['change'].apply(lambda x: 1 if x > 0.03 else -1 if x < -0.03 else 0 ) #threshold 적용하면 48개 남음
minutes_w_rate_3bp = minutes_w_rate_3bp[minutes_w_rate_3bp['label'] != 0]
minutes_w_rate_3bp = minutes_w_rate_3bp[['token_ngram', 'label']]
minutes_w_rate_3bp.shape

(48, 2)

In [124]:
minutes_w_rate.to_csv('minutes_w_rate.csv', encoding='utf-8')
minutes_w_rate_3bp.to_csv('minutes_w_rate_3bp.csv', encoding='utf-8')

#### 뉴스기사

In [7]:
news = pd.read_csv('news_date_ngram.csv', header=0, index_col=[0])
news

,date,token_ngram
0,2005-01-01,"만기/NNG,국채/NNG,수익률/NNG,fed/NNG,fed/NNG,금리/NNG,인..."
1,2005-01-01,"만기/NNG,국채/NNG,가격/NNG,최악/NNG,보내/VV,만기/NNG,국채/NN..."
2,2005-01-01,"마지막/NNG,거래일/NNG,뉴욕/NNG,환시/NNG,달러/NNG,개장/NNG,엔/..."
3,2005-01-01,"마지막/NNG,날/NNG,뉴욕/NNG,금융시장/NNG,한산/NNG,거래/NNG,속/..."
4,2005-01-02,"주도/NNG,연속/NNG,대비/NNG,상승/NNG,하/XSV,대표지수/NNG,최고/..."
...,...,...
253022,2017-12-31,"절상/NNG,중/NNG,실질가치/NNG,오르/VV,중/NNG,기자/NNG,보호무역주..."
253023,2017-12-31,"자료/NNG,일자/NNG,최고/NNG,금리/NNG,박의/NNG,기자/NNG,시장/N..."
253024,2017-12-31,"뉴욕/NNG,은실/NNG,특파원/NNG,거래/NNG,주/NNG,다시/MAG,경제/N..."
253025,2017-12-31,"판매/NNG,정기/NNG,예금/NNG,중/NNG,금리/NNG,정기/NNG,예금/NN..."


In [8]:
news['date'] = news['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d")) #str형식의 date를 date형식으로 변환
news

,date,token_ngram
0,2005-01-01,"만기/NNG,국채/NNG,수익률/NNG,fed/NNG,fed/NNG,금리/NNG,인..."
1,2005-01-01,"만기/NNG,국채/NNG,가격/NNG,최악/NNG,보내/VV,만기/NNG,국채/NN..."
2,2005-01-01,"마지막/NNG,거래일/NNG,뉴욕/NNG,환시/NNG,달러/NNG,개장/NNG,엔/..."
3,2005-01-01,"마지막/NNG,날/NNG,뉴욕/NNG,금융시장/NNG,한산/NNG,거래/NNG,속/..."
4,2005-01-02,"주도/NNG,연속/NNG,대비/NNG,상승/NNG,하/XSV,대표지수/NNG,최고/..."
...,...,...
253022,2017-12-31,"절상/NNG,중/NNG,실질가치/NNG,오르/VV,중/NNG,기자/NNG,보호무역주..."
253023,2017-12-31,"자료/NNG,일자/NNG,최고/NNG,금리/NNG,박의/NNG,기자/NNG,시장/N..."
253024,2017-12-31,"뉴욕/NNG,은실/NNG,특파원/NNG,거래/NNG,주/NNG,다시/MAG,경제/N..."
253025,2017-12-31,"판매/NNG,정기/NNG,예금/NNG,중/NNG,금리/NNG,정기/NNG,예금/NN..."


In [10]:
call_rate.columns = ['date', 'call_rate']
news_w_rate = pd.merge(news, call_rate, how = 'left', on='date')

call_rate.columns = ['before_month', 'call_rate']
news_w_rate['before_month'] = news_w_rate['date'].apply(lambda x: x - timedelta(days=28)) #한달 전 날짜 생성해서 콜금리와 병합
news_w_rate = pd.merge(news_w_rate, call_rate, how = 'left', on='before_month')
news_w_rate

,date,token_ngram,call_rate_x,before_month,call_rate_y
0,2005-01-01,"만기/NNG,국채/NNG,수익률/NNG,fed/NNG,fed/NNG,금리/NNG,인...",3.29,2004-12-04,3.26
1,2005-01-01,"만기/NNG,국채/NNG,가격/NNG,최악/NNG,보내/VV,만기/NNG,국채/NN...",3.29,2004-12-04,3.26
2,2005-01-01,"마지막/NNG,거래일/NNG,뉴욕/NNG,환시/NNG,달러/NNG,개장/NNG,엔/...",3.29,2004-12-04,3.26
3,2005-01-01,"마지막/NNG,날/NNG,뉴욕/NNG,금융시장/NNG,한산/NNG,거래/NNG,속/...",3.29,2004-12-04,3.26
4,2005-01-02,"주도/NNG,연속/NNG,대비/NNG,상승/NNG,하/XSV,대표지수/NNG,최고/...",3.29,2004-12-05,3.26
...,...,...,...,...,...
253022,2017-12-31,"절상/NNG,중/NNG,실질가치/NNG,오르/VV,중/NNG,기자/NNG,보호무역주...",1.58,2017-12-03,1.48
253023,2017-12-31,"자료/NNG,일자/NNG,최고/NNG,금리/NNG,박의/NNG,기자/NNG,시장/N...",1.58,2017-12-03,1.48
253024,2017-12-31,"뉴욕/NNG,은실/NNG,특파원/NNG,거래/NNG,주/NNG,다시/MAG,경제/N...",1.58,2017-12-03,1.48
253025,2017-12-31,"판매/NNG,정기/NNG,예금/NNG,중/NNG,금리/NNG,정기/NNG,예금/NN...",1.58,2017-12-03,1.48


In [8]:
news_w_rate['change'] = news_w_rate['call_rate_x'] - news_w_rate['call_rate_y']
news_w_rate_3bp = news_w_rate

news_w_rate['label'] = news_w_rate['change'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0 ) #threshold 3bp를 적용하지 않으면 114개 남음
news_w_rate = news_w_rate[news_w_rate['label'] != 0]
news_w_rate = news_w_rate[['token_ngram', 'label']]
news_w_rate['label']

NameError: name 'news_w_rate' is not defined

In [12]:
news_w_rate_3bp['label'] = news_w_rate_3bp['change'].apply(lambda x: 1 if x > 0.03 else -1 if x < -0.03 else 0 ) #threshold 적용하면 48개 남음
news_w_rate_3bp = news_w_rate_3bp[news_w_rate_3bp['label'] != 0]
news_w_rate_3bp = news_w_rate_3bp[['token_ngram', 'label']]
news_w_rate_3bp.shape

(94660, 2)

In [13]:
news_w_rate.to_csv('news_w_rate.csv', encoding='utf-8')
news_w_rate_3bp.to_csv('news_w_rate_3bp.csv', encoding='utf-8')

#### 채권 분석 보고서

In [61]:
bond_report = pd.read_csv('bond_report_ngram_date.csv', header=0, index_col=[0])
bond_report.columns = ['date', 'token_ngram']
bond_report['date'] = bond_report['date'].apply(lambda x: datetime.strptime(x, "%y.%m.%d")) #str형식의 date를 date형식으로 변환
bond_report

,date,token_ngram
0,2017-12-27,"프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN..."
1,2017-12-26,"금융/NNG,수급/NNG,영향/NNG,점검/NNG,채권/NNG,보/VV,조사/NNG..."
2,2017-12-20,"의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,..."
3,2017-12-20,"속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,..."
4,2017-12-18,"fed/NNG,성장/NNG,물가/NNG,fed/NNG,ff/NNG,목표/NNG,금리..."
...,...,...
3092,2008-04-21,"믹스/NNG,합의/NNG,반영/NNG,예상/NNG,경제/NNG,성장률/NNG,하향/..."
3093,2008-04-14,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무..."
3094,2008-04-07,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무..."
3095,2008-04-07,"가/VV,채권투자/NNG,채권시장/NNG,전망/NNG,금융시장/NNG,차/NNG,트..."


In [62]:
bond_w_rate = pd.merge(bond_report, call_rate, how = 'left', on='date')
bond_w_rate

,date,token_ngram,call_rate
0,2017-12-27,"프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...",1.51
1,2017-12-26,"금융/NNG,수급/NNG,영향/NNG,점검/NNG,채권/NNG,보/VV,조사/NNG...",1.52
2,2017-12-20,"의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...",1.51
3,2017-12-20,"속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...",1.51
4,2017-12-18,"fed/NNG,성장/NNG,물가/NNG,fed/NNG,ff/NNG,목표/NNG,금리...",1.50
...,...,...,...
3092,2008-04-21,"믹스/NNG,합의/NNG,반영/NNG,예상/NNG,경제/NNG,성장률/NNG,하향/...",4.97
3093,2008-04-14,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무...",5.00
3094,2008-04-07,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무...",5.04
3095,2008-04-07,"가/VV,채권투자/NNG,채권시장/NNG,전망/NNG,금융시장/NNG,차/NNG,트...",5.04


In [64]:
call_rate.columns = ['before_month', 'call_rate']
bond_w_rate['before_month'] = bond_w_rate['date'].apply(lambda x: x - timedelta(days=28)) #한달 전 날짜 생성해서 콜금리와 병합
bond_w_rate = pd.merge(bond_w_rate, call_rate, how = 'left', on='before_month')
bond_w_rate

,date,token_ngram,call_rate_x,before_month,call_rate_y
0,2017-12-27,"프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...",1.51,2017-11-29,1.26
1,2017-12-26,"금융/NNG,수급/NNG,영향/NNG,점검/NNG,채권/NNG,보/VV,조사/NNG...",1.52,2017-11-28,1.26
2,2017-12-20,"의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...",1.51,2017-11-22,1.24
3,2017-12-20,"속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...",1.51,2017-11-22,1.24
4,2017-12-18,"fed/NNG,성장/NNG,물가/NNG,fed/NNG,ff/NNG,목표/NNG,금리...",1.50,2017-11-20,1.25
...,...,...,...,...,...
3092,2008-04-21,"믹스/NNG,합의/NNG,반영/NNG,예상/NNG,경제/NNG,성장률/NNG,하향/...",4.97,2008-03-24,4.97
3093,2008-04-14,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무...",5.00,2008-03-17,4.96
3094,2008-04-07,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무...",5.04,2008-03-10,4.99
3095,2008-04-07,"가/VV,채권투자/NNG,채권시장/NNG,전망/NNG,금융시장/NNG,차/NNG,트...",5.04,2008-03-10,4.99


In [79]:
bond_w_rate['change'] = bond_w_rate['call_rate_x'] - bond_w_rate['call_rate_y']
bond_w_rate['label'] = bond_w_rate['change'].apply(lambda x: 1 if x > 0 else -1 if x < 0 else 0 ) #threshold 3bp를 적용하지 않으면 2368개 남음
bond_w_rate = bond_w_rate[bond_w_rate['label'] != 0]
bond_w_rate = bond_w_rate[['token_ngram', 'label']]
bond_w_rate

,token_ngram,label
0,"프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...",1
1,"금융/NNG,수급/NNG,영향/NNG,점검/NNG,채권/NNG,보/VV,조사/NNG...",1
2,"의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...",1
3,"속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...",1
4,"fed/NNG,성장/NNG,물가/NNG,fed/NNG,ff/NNG,목표/NNG,금리...",1
...,...,...
3091,"경기/NNG,전망/NNG,비교적/VAX,분명/MAG,인플레이션/NNG,명분/NNG,...",-1
3093,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무...",1
3094,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무...",1
3095,"가/VV,채권투자/NNG,채권시장/NNG,전망/NNG,금융시장/NNG,차/NNG,트...",1


In [80]:
bond_w_rate_3bp = bond_w_rate
bond_w_rate_3bp['label'] = bond_w_rate_3bp['change'].apply(lambda x: 1 if x > 0.03 else -1 if x < -0.03 else 0 ) #threshold 적용하면 881개 남음
bond_w_rate_3bp = bond_w_rate_3bp[bond_w_rate['label'] != 0]
bond_w_rate_3bp = bond_w_rate_3bp[['token_ngram', 'label']]
bond_w_rate_3bp

,token_ngram,label
0,"프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...",1
1,"금융/NNG,수급/NNG,영향/NNG,점검/NNG,채권/NNG,보/VV,조사/NNG...",1
2,"의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...",1
3,"속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...",1
4,"fed/NNG,성장/NNG,물가/NNG,fed/NNG,ff/NNG,목표/NNG,금리...",1
...,...,...
3090,"fed/NNG,원자재/NNG,가격/NNG,상승/NNG,둔화/NNG,여부/NNG,관심...",-1
3091,"경기/NNG,전망/NNG,비교적/VAX,분명/MAG,인플레이션/NNG,명분/NNG,...",-1
3093,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무...",1
3094,"돌아온/VV,박스권/NNG,하단/NNG,상단/NNG,테스트/NNG,가능성/NNG,무...",1


In [81]:
bond_w_rate.to_csv('bond_w_rate.csv', encoding='utf-8')
bond_w_rate_3bp.to_csv('bond_w_rate_3bp.csv', encoding='utf-8')